In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from shrinkage_estimators import precision_shrinkage

%matplotlib inline

In [2]:
DATA_PATH = "data"
OUTPUT_PATH = "output"
NSAMP = 21
NDIM = 18

In [10]:
for P_fname in [f for f in os.listdir(DATA_PATH) if not f.startswith('.')]:
    # Load mock power spectrum data
    P = np.loadtxt(os.path.join(DATA_PATH, P_fname))
    
    # Target
    target = np.identity(NDIM)
    
    # Sample covariance matrix
    S = np.cov(P)
    
    # Shrinkage estimation of the precision matrix
    Pi_shrinkage, alpha, beta = precision_shrinkage.shrinkage(P, target)
    evals = np.linalg.eigvalsh(Pi_shrinkage)
    print("Version", v, "Smallest (most negative) eigenvalue:", np.min(evals))
    
    # Extract mock power spectrum version number from P_fname
    v = int((P_fname.split(".")[0]).split("v")[1])
    
    # Save precision shrinkage estimate to file
    out_fname = f"prec_shrinkage_{NDIM}_{NDIM}_v{v}.matrix"
    np.savetxt(os.path.join(OUTPUT_PATH, out_fname),
               Pi_shrinkage,
               header=f"alpha={alpha}, beta={beta}")

Version 60 Smallest (most negative) eigenvalue: -1.6862876479279483e-06
Version 41 Smallest (most negative) eigenvalue: -1.2948942587471765e-06
Version 30 Smallest (most negative) eigenvalue: -1.5528540291226177e-06
Version 18 Smallest (most negative) eigenvalue: -3.789234447426443e-06
Version 77 Smallest (most negative) eigenvalue: -2.5566726062811484e-06
Version 69 Smallest (most negative) eigenvalue: -6.612732959530321e-07
Version 32 Smallest (most negative) eigenvalue: -1.348021252589167e-06
Version 3 Smallest (most negative) eigenvalue: -9.282645742889132e-07
Version 15 Smallest (most negative) eigenvalue: -1.148022259087586e-06
Version 23 Smallest (most negative) eigenvalue: -3.117922381454224e-06
Version 78 Smallest (most negative) eigenvalue: -2.866463118936177e-06
Version 33 Smallest (most negative) eigenvalue: -1.692553942625742e-06
Version 6 Smallest (most negative) eigenvalue: -1.0409046619155858e-06
Version 13 Smallest (most negative) eigenvalue: -1.8582727354017447e-06
Ve